## Problem 1

In [3]:
from scipy.optimize import linprog

In [4]:
def nums_from_button_line(line):
    x = line[10:].split(', ')
    return (int(x[0][2:]), int(x[1][2:]))

def nums_from_prize_line(line):
    x = line[7:].split(', ')
    return (int(x[0][2:]), int(x[1][2:]))

In [14]:
def solve1(input_filename):
    f = open(f'../inputs/{input_filename}.txt', 'r')

    tokens = 0
    while True:
        line_A = f.readline()
        line_B = f.readline()
        line_prize = f.readline()
        f.readline()

        if line_A == '':
            break
        (a1, a2) = nums_from_button_line(line_A)
        (b1, b2) = nums_from_button_line(line_B)
        (p1, p2) = nums_from_prize_line(line_prize)

        res = linprog(c=[3, 1], A_eq=[[a1, b1], [a2, b2]], b_eq=[p1, p2], bounds=(0, 100), integrality=1)
        if res.status == 2:
            continue
        if res.status != 0:
            print(a1, a2, b1, b2, p1, p2)
            raise Exception('Unknown error!')
        tokens += int(res.mip_dual_bound)
    f.close()
    return tokens

## Problem 2

### Bad solution

This is the same as before, but here it's not giving the correct answer:

In [47]:
# Not working, I don't understand why :/
def solve2(input_filename):
    f = open(f'../inputs/{input_filename}.txt', 'r')
    big_number = 10000000000000

    tokens = 0
    while True:
        line_A = f.readline()
        line_B = f.readline()
        line_prize = f.readline()
        f.readline()

        if line_A == '':
            break
        (a1, a2) = nums_from_button_line(line_A)
        (b1, b2) = nums_from_button_line(line_B)
        (p1, p2) = nums_from_prize_line(line_prize)

        res = linprog(c=[3, 1], A_eq=[[a1, b1], [a2, b2]], b_eq=[big_number+p1, big_number+p2], bounds=(0, None), integrality=1)
        if res.status == 2:
            continue
        if res.status != 0:
            print(a1, a2, b1, b2, p1, p2)
            raise Exception('This should not happen!')
        tokens += int(res.mip_dual_bound)
    f.close()
    return tokens

### Good solution

In [1]:
import numpy as np

In [50]:
def check_solution(x, A, p):
    return np.all(p == np.matmul(A, x))

def solve2(input_filename):
    f = open(f'../inputs/{input_filename}.txt', 'r')
    big_num = 10000000000000
    tokens = 0

    while True:
        line_A = f.readline()
        line_B = f.readline()
        line_prize = f.readline()
        f.readline()

        if line_A == '':
            break
        (a1, a2) = nums_from_button_line(line_A)
        (b1, b2) = nums_from_button_line(line_B)
        (p1, p2) = nums_from_prize_line(line_prize)

        A = np.array([[a1, b1], [a2, b2]])
        p =  np.array([p1+big_num, p2+big_num])

        res = np.linalg.solve(A, p)
        x = np.rint(res)
        if np.any(x < 0) or not check_solution(x, A, p):
            continue
        tokens += 3*x[0] + x[1]
    f.close()
    return tokens